#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Financial Stress Test Optimization

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Financial Data

In [ ]:
# Historical data for a few stocks
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'AMD', 'INTC']
periods = [{'start': '2005-01-01', 'end': '2023-12-31'}]

data = pd.DataFrame()
for ticker in tickers:
    try:
        stock_data = yf.download(ticker, start='2005-01-01', end='2023-12-31')['Close']
        stock_data = stock_data.dropna()
        data[ticker] = stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Align data to have the same time index for all stocks
data = data.dropna()

# Calculate returns
returns = data.pct_change().dropna()

print(returns.head())
# Save to CSV for reference
returns.to_csv('downturn_data.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                AAPL     GOOGL      MSFT      AMZN      META      TSLA  \
Date                                                                     
2012-05-21  0.058260  0.022835  0.016399  0.019921 -0.109861  0.043904   
2012-05-22 -0.007679 -0.021674  0.000336 -0.012746 -0.089039  0.070559   
2012-05-23  0.024400  0.014414 -0.021841  0.009056  0.032258  0.007143   
2012-05-24 -0.009184 -0.009517 -0.001374 -0.009389  0.032187 -0.023855   
2012-05-25 -0.005360 -0.020094 -0.000344 -0.010918 -0.033909 -0.015522   

                NFLX      NVDA       AMD      INTC  
Date                                                
2012-05-21  0.025443  0.017384  0.048253  0.003069  
2012-05-22 -0.056175 -0.012205 -0.022222 -0.004589  
2012-05-23  0.062029  0.024712 -0.012987 -0.022666  
2012-05-24 -0.022806 -0.026528 -0.009868  0.008255  
2012-05-25 -0.000711  0.023947  0.033223  0.003509  


In [ ]:
# Load the data
returns = pd.read_csv('downturn_data.csv', index_col=0)

# Ensure all columns are numeric again after reloading
returns = returns.apply(pd.to_numeric, errors='coerce').dropna(axis=1)
returns.head()

,AAPL,GOOGL,MSFT,AMZN,META,TSLA,NFLX,NVDA,AMD,INTC
Date,,,,,,,,,,
2012-05-21,0.058260,0.022835,0.016399,0.019921,-0.109861,0.043904,0.025443,0.017384,0.048253,0.003069
2012-05-22,-0.007679,-0.021674,0.000336,-0.012746,-0.089039,0.070559,-0.056175,-0.012205,-0.022222,-0.004589
2012-05-23,0.024400,0.014414,-0.021841,0.009056,0.032258,0.007143,0.062029,0.024712,-0.012987,-0.022666
2012-05-24,-0.009184,-0.009517,-0.001374,-0.009389,0.032187,-0.023855,-0.022806,-0.026528,-0.009868,0.008255
2012-05-25,-0.005360,-0.020094,-0.000344,-0.010918,-0.033909,-0.015522,-0.000711,0.023947,0.033223,0.003509


#Optimization Model

In [ ]:
# Model setup
assets = list(returns.columns)
n = len(assets)

# Create Pyomo model
model1 = ConcreteModel()
model1.assets = RangeSet(n)

# Calculate means and absolute deviations
returns_mean = returns.mean().values
absolute_deviations = np.abs(returns - returns_mean).mean().values
annual_returns = (1 + returns.mean().values) ** 252 - 1  # Assuming 252 trading days in a year

# Make parameters mutable
model1.mu = Param(model1.assets, mutable=True, initialize=lambda model1, i: returns_mean[i-1])
model1.dev = Param(model1.assets, mutable=True, initialize=lambda model1, i: absolute_deviations[i-1])
model1.annual_return = Param(model1.assets, mutable=True, initialize=lambda model1, i: annual_returns[i-1])

total_budget = 1000000 # Example budget of $1,000,000

# Decision variables
model1.x = Var(model1.assets, domain=NonNegativeReals)
model1.y = Var(model1.assets, domain=Binary)

# Objective: Minimize mean absolute deviation Maximize annual return
def objective_rule(model1):
    return sum(model1.x[i] * model1.dev[i] for i in model1.assets)

def annual_return_objective_rule(model1):
    return sum(model1.x[i] * model1.annual_return[i] for i in model1.assets)

# Combine objectives using scalarization (e.g., weighted sum)
weight_abs_dev = 0.5  # Adjustable weight for absolute deviation
weight_ann_return = 0.5  # Adjustable weight for annual return

def combined_objective_rule(model1):
    return (weight_abs_dev * sum(model1.x[i] * model1.dev[i] for i in model1.assets) -
            weight_ann_return * sum(model1.x[i] * model1.annual_return[i] for i in model1.assets))
model1.combined_objective = Objective(rule=combined_objective_rule, sense=minimize)

# Constraints
def percent_constraint_rule(model1):
    return sum(model1.x[i] for i in model1.assets) == 1
model1.percent_constraint = Constraint(rule=percent_constraint_rule)

def risk_tolerance_rule(model1):
    return sum(model1.mu[i] * model1.x[i] for i in model1.assets) >= 0.001  # Lowered threshold
model1.risk_tolerance = Constraint(rule=risk_tolerance_rule)

def diversification_constraint_rule(model1, i):
    return model1.x[i] <= 0.4  # Loosened constraint
model1.diversification_constraint = Constraint(model1.assets, rule=diversification_constraint_rule)

def hedging_effectiveness_rule(model1, i):
    return model1.x[i] >= 0.03 * model1.y[i]  # Loosened constraint
model1.hedging_effectiveness = Constraint(model1.assets, rule=hedging_effectiveness_rule)

model1.pprint()

1 RangeSet Declarations
    assets : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

3 Param Declarations
    annual_return : Size=10, Index=assets, Domain=Any, Default=None, Mutable=True
        Key : Value
          1 : 0.28996147591300714
          2 :  0.2571785003903089
          3 :  0.3154363327301377
          4 :  0.3244112640205663
          5 : 0.31322363722484003
          6 :  0.7886189634885505
          7 :  0.5696471132770096
          8 :  0.7220586374131297
          9 :  0.5519568146794993
         10 : 0.14444480632322887
    dev : Size=10, Index=assets, Domain=Any, Default=None, Mutable=True
        Key : Value
          1 : 0.012523649534326772
          2 : 0.011725438418783832
          3 : 0.011507961742998115
          4 : 0.014072128688754352
          5 :  0.01638881129341217
          6 : 0.024662728949074803
          7 : 0.019460215137115763
          8 : 0.018969712421723767
          9 :  0.0248899686118

In [ ]:
# Solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5
opt.options['ratioGap'] = .01
results = opt.solve(model1, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmphdrn6823.pyomo.lp -stat=1 -solve -solu /tmp/tmphdrn6823.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Ra

In [ ]:
# Print optimal allocations
print("Optimal Allocations:")
allocations = {assets[i-1]: model1.x[i].value for i in model1.assets}
for asset, allocation in allocations.items():
    print(f"{asset}: {allocation:.4f}")

# Calculate and print the mean absolute deviation
mean_absolute_deviation = sum(model1.x[i].value * model1.dev[i] for i in model1.assets)
mean_absolute_deviation_value = value(mean_absolute_deviation)
print(f"\nMean Absolute Deviation: {mean_absolute_deviation_value:.4f}")

# Calculate and print the mean annual return
mean_annual_return = sum(model1.x[i].value * model1.annual_return[i] for i in model1.assets)
mean_annual_return_value = value(mean_annual_return)
print(f"Mean Annual Return: {mean_annual_return_value:.4f}")

Optimal Allocations:
AAPL: 0.0000
GOOGL: 0.0000
MSFT: 0.0000
AMZN: 0.0000
META: 0.0000
TSLA: 0.4000
NFLX: 0.2000
NVDA: 0.4000
AMD: 0.0000
INTC: 0.0000

Mean Absolute Deviation: 0.0213
Mean Annual Return: 0.7182


#Scenario Analysis - Optimization Model

In [ ]:
scenarios = {
    'recession': {'market_drop': 0.1, 'interest_rate': 0.03, 'inflation': 0.02, 'regulation_factor': 0.85},
    'market_crash': {'market_drop': 0.2, 'interest_rate': 0.02, 'inflation': 0.02, 'regulation_factor': 0.7},
    'economic_slowdown': {'market_drop': 0.05, 'interest_rate': 0.025, 'inflation': 0.015, 'regulation_factor': 0.9},
    'high_interest_rate': {'market_drop': 0.1, 'interest_rate': 0.05, 'inflation': 0.03, 'regulation_factor': 0.8},
    'low_interest_rate': {'market_drop': 0.03, 'interest_rate': 0.015, 'inflation': 0.01, 'regulation_factor': 0.95},
    'inflation': {'market_drop': 0.1, 'interest_rate': 0.04, 'inflation': 0.05, 'regulation_factor': 0.8},
    'deflation': {'market_drop': 0.03, 'interest_rate': 0.01, 'inflation': -0.01, 'regulation_factor': 1.0},
    'regulatory_changes': {'market_drop': 0.05, 'interest_rate': 0.03, 'inflation': 0.02, 'regulation_factor': 0.75},
    'risk_averse': {'market_drop': 0.03, 'interest_rate': 0.04, 'inflation': 0.02, 'regulation_factor': 0.5},
    'risk_seeking': {'market_drop': 0.15, 'interest_rate': 0.025, 'inflation': 0.03, 'regulation_factor': 1.2}
}

# Define a function to update model parameters based on a scenario
def apply_scenario(model1, scenario_params):
    market_drop = scenario_params['market_drop']
    interest_rate = scenario_params['interest_rate']
    inflation = scenario_params['inflation']
    regulation_factor = scenario_params['regulation_factor']

    # Adjust returns to simulate market drop
    adjusted_returns_mean = (1 - market_drop) * returns.mean().values
    for i in model1.assets:
        model1.mu[i] = adjusted_returns_mean[i-1]

    # Adjust annual returns similarly
    adjusted_annual_returns = (1 + adjusted_returns_mean) ** 252 - 1  # Annualized returns
    for i in model1.assets:
        model1.annual_return[i] = adjusted_annual_returns[i-1]

    # Adjust regulation factor for diversification constraints
    def adjusted_diversification_constraint_rule(model1, i):
        return model1.x[i] <= 0.4 * regulation_factor
    model1.diversification_constraint = Constraint(model1.assets, rule=adjusted_diversification_constraint_rule)

# Define weight adjustment function to update parameters
def adjust_weights_for_scenario(scenario_name):
    if scenario_name == 'risk_averse':
        return 0.8, 0.2  # More focus on minimizing risk
    elif scenario_name == 'risk_seeking':
        return 0.2, 0.8  # More focus on maximizing returns
    else:
        return 0.5, 0.5  # Balanced approach

# Apply weight adjustments
def apply_scenario_with_weights(model1, scenario_name, scenario_params):
    apply_scenario(model1, scenario_params)

    weight_abs_dev, weight_ann_return = adjust_weights_for_scenario(scenario_name)

    def combined_objective_rule(model1):
        return (weight_abs_dev * sum(model1.x[i] * model1.dev[i] for i in model1.assets) -
                weight_ann_return * sum(model1.x[i] * model1.annual_return[i] for i in model1.assets))

    model1.combined_objective = Objective(rule=combined_objective_rule, sense=minimize)

model1.pprint()

1 RangeSet Declarations
    assets : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

3 Param Declarations
    annual_return : Size=10, Index=assets, Domain=Any, Default=None, Mutable=True
        Key : Value
          1 :  0.2416447314807575
          2 :  0.2147674804567048
          3 :  0.2624599642009635
          4 : 0.26977890202242083
          5 : 0.26065439658661504
          6 :  0.6393711180095569
          7 :  0.4670808377601796
          8 :  0.5873517855226198
          9 : 0.45301100008785133
         10 : 0.12152155368594375
    dev : Size=10, Index=assets, Domain=Any, Default=None, Mutable=True
        Key : Value
          1 : 0.012523649534326772
          2 : 0.011725438418783832
          3 : 0.011507961742998115
          4 : 0.014072128688754352
          5 :  0.01638881129341217
          6 : 0.024662728949074803
          7 : 0.019460215137115763
          8 : 0.018969712421723767
          9 :  0.0248899686118

In [ ]:
# Solve the model for each scenario and collect results
scenario_results = {}
for scenario_name, scenario_params in scenarios.items():
    print(f"Applying scenario: {scenario_name}")

    # Apply scenario adjustments and weights
    apply_scenario_with_weights(model1, scenario_name, scenario_params)

    # Solve the model with CBC solver
    opt = SolverFactory('cbc')
    opt.options['seconds'] = 5
    opt.options['ratioGap'] = 0.01
    results = opt.solve(model1, tee=True)

    # Collect results
    optimal_allocations = {f"Asset {i}": model1.x[i].value for i in model1.assets}
    mean_absolute_deviation = sum(model1.x[i].value * model1.dev[i] for i in model1.assets)
    mean_annual_return = sum(model1.x[i].value * model1.annual_return[i] for i in model1.assets)

    # Store results for the scenario
    scenario_results[scenario_name] = {
        'allocations': optimal_allocations,
        'mean_absolute_deviation': value(mean_absolute_deviation),
        'mean_annual_return': value(mean_annual_return)
    }

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Applying scenario: recession
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmp2g7xj0ok.pyomo.lp -stat=1 -solve -solu /tmp/tmp2g7xj0ok.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Applying scenario: market_crash
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpy9q9pra1.pyomo.lp -stat=1 -solve -solu /tmp/tmpy9q9pra1.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L 1.0, 

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


...

Applying scenario: regulatory_changes
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpg0and40g.pyomo.lp -stat=1 -solve -solu /tmp/tmpg0and40g.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Applying scenario: risk_averse
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpfxv2jfrf.pyomo.lp -stat=1 -solve -solu /tmp/tmpfxv2jfrf.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L 1.0, 


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Applying scenario: risk_seeking
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpy63hh5i4.pyomo.lp -stat=1 -solve -solu /tmp/tmpy63hh5i4.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
Presolve 2 (-20) rows, 10 (-10) columns and 20 (-30) elements
Statistics for presolved model
Original problem has 10 integers (10 of which binary)


Problem has 2 rows, 10 columns (10 with objective) and 20 elements
Column breakdown:
0 of type 0.0->inf, 10 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 1 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
1 of type G other, 0 of type L 0.0, 0 of type L 1.0, 

In [ ]:
# Print results for each scenario
for scenario_name, result in scenario_results.items():
    print(f"\nResults for Scenario: {scenario_name}")

    # Print allocations with asset names
    print("Optimal Allocations:")
    for i, allocation in enumerate(result['allocations'].values(), start=1):
        asset_name = assets[i-1]  # Use the asset name from the list
        print(f"  {asset_name}: {allocation:.4f}")

    # Print objective values
    print(f"Mean Absolute Deviation: {result['mean_absolute_deviation']:.4f}")
    print(f"Mean Annual Return: {result['mean_annual_return']:.4f}")


Results for Scenario: recession
Optimal Allocations:
  AAPL: 0.0000
  GOOGL: 0.0000
  MSFT: 0.0000
  AMZN: 0.0000
  META: 0.0000
  TSLA: 0.3400
  NFLX: 0.3200
  NVDA: 0.3400
  AMD: 0.0000
  INTC: 0.0000
Mean Absolute Deviation: 0.0211
Mean Annual Return: 0.6085

Results for Scenario: market_crash
Optimal Allocations:
  AAPL: 0.0000
  GOOGL: 0.0000
  MSFT: 0.0000
  AMZN: 0.0000
  META: 0.0000
  TSLA: 0.2800
  NFLX: 0.2800
  NVDA: 0.2800
  AMD: 0.1600
  INTC: 0.0000
Mean Absolute Deviation: 0.0216
Mean Annual Return: 0.5075

Results for Scenario: economic_slowdown
Optimal Allocations:
  AAPL: 0.0000
  GOOGL: 0.0000
  MSFT: 0.0000
  AMZN: 0.0000
  META: 0.0000
  TSLA: 0.3600
  NFLX: 0.2800
  NVDA: 0.3600
  AMD: 0.0000
  INTC: 0.0000
Mean Absolute Deviation: 0.0212
Mean Annual Return: 0.6585

Results for Scenario: high_interest_rate
Optimal Allocations:
  AAPL: 0.0000
  GOOGL: 0.0000
  MSFT: 0.0000
  AMZN: 0.0000
  META: 0.0000
  TSLA: 0.3200
  NFLX: 0.3200
  NVDA: 0.3200
  AMD: 0.0400
  

#Baseline Allocations

In [ ]:
# Allow the user to pick assets (can be customized based on the desired tickers/assets)
chosen_assets = ['AAPL', 'META', 'NVDA', 'MSFT']  # Example list of selected assets

# Filter the returns DataFrame to include only the chosen assets
filtered_returns = returns[chosen_assets]

# Calculate daily mean returns for each stock
daily_mean_returns = filtered_returns.mean()

# Calculate mean absolute deviation (MAD) for each stock
filtered_returns_mean = filtered_returns.mean().values
absolute_deviations = np.abs(filtered_returns - filtered_returns_mean).mean().values

# Calculate annualized return for each stock
annual_returns = (1 + daily_mean_returns) ** 252 - 1  # Assuming 252 trading days in a year

# Portfolio allocation (25% of $1,000,000 to each stock)
portfolio_allocation = 1000000 / 4  # Equal allocation for 4 stocks

# Weighted MAD and annual returns (since allocation is equal, weights are the same)
weights = [0.25, 0.25, 0.25, 0.25]  # 25% for each stock
combined_mad = sum(w * absolute_deviations[i] for i, w in enumerate(weights))
combined_annual_return = sum(w * annual_returns[i] for i, w in enumerate(weights))

# Display results
print("Combined Portfolio Results:")
print(f"Combined Mean Absolute Deviation (MAD): {combined_mad:.6f}")
print(f"Combined Annual Return: {combined_annual_return:.6f}")

# Print results
print(results)

Combined Portfolio Results:
Combined Mean Absolute Deviation (MAD): 0.014848
Combined Annual Return: 0.410170

Problem: 
- Name: unknown
  Lower bound: -0.48166379
  Upper bound: -0.48166379
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.034604787826538086
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



<ipython-input-101-5d805bc07731>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_annual_return = sum(w * annual_returns[i] for i, w in enumerate(weights))


#Scenario Analysis on Baseline Model

In [ ]:
# Allow the user to pick assets (can be customized based on the desired tickers/assets)
chosen_assets = ['AAPL', 'META', 'NVDA', 'MSFT']  # Example list of selected assets

# Filter the returns DataFrame to include only the chosen assets
filtered_returns = returns[chosen_assets]

# Calculate daily mean returns for each stock
daily_mean_returns = filtered_returns.mean()

# Calculate mean absolute deviation (MAD) for each stock
filtered_returns_mean = filtered_returns.mean().values
absolute_deviations = np.abs(filtered_returns - filtered_returns_mean).mean().values

# Calculate annualized return for each stock
annual_returns = (1 + daily_mean_returns) ** 252 - 1  # Assuming 252 trading days in a year

# Portfolio allocation (25% of $1,000,000 to each stock)
portfolio_allocation = 1000000 / 4  # Equal allocation for 4 stocks

# Weighted MAD and annual returns (since allocation is equal, weights are the same)
weights = [0.25, 0.25, 0.25, 0.25]  # 25% for each stock
initial_mad = sum(w * absolute_deviations[i] for i, w in enumerate(weights))
initial_annual_return = sum(w * annual_returns[i] for i, w in enumerate(weights))

# Scenarios
scenarios = {
    'recession': {'market_drop': 0.1, 'interest_rate': 0.03, 'inflation': 0.02, 'regulation_factor': 0.85},
    'market_crash': {'market_drop': 0.2, 'interest_rate': 0.02, 'inflation': 0.02, 'regulation_factor': 0.7},
    'economic_slowdown': {'market_drop': 0.05, 'interest_rate': 0.025, 'inflation': 0.015, 'regulation_factor': 0.9},
    'high_interest_rate': {'market_drop': 0.1, 'interest_rate': 0.05, 'inflation': 0.03, 'regulation_factor': 0.8},
    'low_interest_rate': {'market_drop': 0.03, 'interest_rate': 0.015, 'inflation': 0.01, 'regulation_factor': 0.95},
    'inflation': {'market_drop': 0.1, 'interest_rate': 0.04, 'inflation': 0.05, 'regulation_factor': 0.8},
    'deflation': {'market_drop': 0.03, 'interest_rate': 0.01, 'inflation': -0.01, 'regulation_factor': 1.0},
    'regulatory_changes': {'market_drop': 0.05, 'interest_rate': 0.03, 'inflation': 0.02, 'regulation_factor': 0.75},
    'risk_averse': {'market_drop': 0.03, 'interest_rate': 0.04, 'inflation': 0.02, 'regulation_factor': 0.5},
    'risk_seeking': {'market_drop': 0.15, 'interest_rate': 0.025, 'inflation': 0.03, 'regulation_factor': 1.2}
}

# Function to simulate portfolio performance under each scenario
def simulate_scenario(scenario):
    market_drop = scenario['market_drop']
    regulation_factor = scenario['regulation_factor']
    inflation = scenario['inflation']

    # Adjust annual returns based on the scenario
    adjusted_annual_returns = annual_returns * (1 - market_drop) * regulation_factor
    # Optionally, adjust MAD to reflect volatility (e.g., higher for risky scenarios)
    adjusted_mad = absolute_deviations * (1 + market_drop * 0.5)  # Scale MAD with market drop

    # Calculate combined metrics
    combined_mad = sum(w * adjusted_mad[i] for i, w in enumerate(weights))
    combined_annual_return = sum(w * adjusted_annual_returns[i] for i, w in enumerate(weights))

    return combined_mad, combined_annual_return

# Simulate each scenario
scenario_results = {}
for name, scenario in scenarios.items():
    combined_mad, combined_annual_return = simulate_scenario(scenario)
    scenario_results[name] = {
        'Combined MAD': combined_mad,
        'Combined Annual Return': combined_annual_return
    }

# Convert results to a DataFrame for better readability
scenario_df = pd.DataFrame.from_dict(scenario_results, orient='index')
print(scenario_df)


                    Combined MAD  Combined Annual Return
recession               0.015590                0.313780
market_crash            0.016332                0.229695
economic_slowdown       0.015219                0.350695
high_interest_rate      0.015590                0.295322
low_interest_rate       0.015070                0.377972
inflation               0.015590                0.295322
deflation               0.015070                0.397865
regulatory_changes      0.015219                0.292246
risk_averse             0.015070                0.198932
risk_seeking            0.015961                0.418373


<ipython-input-102-612b72596e1d>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_annual_return = sum(w * annual_returns[i] for i, w in enumerate(weights))
<ipython-input-102-612b72596e1d>:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_annual_return = sum(w * adjusted_annual_returns[i] for i, w in enumerate(weights))
